In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [3]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))

x_letter = train[:,1]
x_letter = np.reshape(x_letter, (-1, 1))
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()

y = train[:,0]
y = np.reshape(y, (-1, 1))
en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(x.shape)
print(x_letter.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 26)
(2048, 10)


In [4]:
valid_size = 48
valid_x = x[-valid_size:]
x = x[:-48]

valid_x_letter = x_letter[-valid_size:]
x_letter = x_letter[:-48]

valid_y = y[-valid_size:]
y = y[:-48]

print(x.shape)
print(valid_x.shape)
print(x_letter.shape)
print(valid_x_letter.shape)
print(y.shape)
print(valid_y.shape)

(2000, 28, 28, 1)
(48, 28, 28, 1)
(2000, 26)
(48, 26)
(2000, 10)
(48, 10)


In [5]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [6]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(6000, 28, 28, 1)


In [7]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

print(y_total.shape)

(6000, 10)


In [8]:
x_letter_total = x_letter.copy()
for i in range(n):
    arr = x_letter.copy()
    x_letter_total = np.concatenate((x_letter_total, arr), axis=0)
    
print(x_letter_total.shape)

(6000, 26)


In [9]:
x_train, x_val, y_train, y_val = train_test_split(x_total, y_total, test_size=0.2, shuffle=True)#, stratify=y_total)
x_letter_train = x_letter_total[:x_train.shape[0],:]
x_letter_val = x_letter_total[x_train.shape[0]:,:]

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)
print(x_letter_train.shape)
print(x_letter_val.shape)

(4800, 28, 28, 1)
(1200, 28, 28, 1)
(4800, 10)
(1200, 10)
(4800, 26)
(1200, 26)


In [10]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (3,3), activation='relu', padding='same')(input1)
#x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
#x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
#x1 = Dropout(0.3)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(26,))
merge = concatenate([x1, input2])

x2 = Dense(500, activation='relu')(merge)
x2 = Dropout(0.3)(x2)
x2 = Dense(100, activation='relu')(x2)
x2 = Dropout(0.3)(x2)
x2 = Dense(50, activation='relu')(x2)
outputs = Dense(10, activation='softmax')(x2)

model = Model(inputs = [input1, input2], outputs = outputs)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 64)   16448       max_pooling2d[0][0]              
__________________________________________________________________________________________________
max_poolin

In [11]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

#es = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1)
cp = ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = model.fit([x_train, x_letter_train], y_train,
                    validation_data=([x_val, x_letter_val], y_val), 
                    batch_size=64, epochs=100, verbose=1, callbacks = [cp])

Train on 4800 samples, validate on 1200 samples
Epoch 1/100
4800/4800 [==============================] - 5s 1ms/step - loss: 2.7785 - acc: 0.1235 - val_loss: 2.2556 - val_acc: 0.1567
Epoch 2/100
4800/4800 [==============================] - 1s 303us/step - loss: 2.2431 - acc: 0.1635 - val_loss: 2.1315 - val_acc: 0.2117
Epoch 3/100
4800/4800 [==============================] - 1s 287us/step - loss: 2.1621 - acc: 0.2023 - val_loss: 2.0048 - val_acc: 0.2933
Epoch 4/100
4800/4800 [==============================] - 1s 279us/step - loss: 2.0204 - acc: 0.2646 - val_loss: 1.8902 - val_acc: 0.3283
Epoch 5/100
4800/4800 [==============================] - 1s 296us/step - loss: 1.8774 - acc: 0.3210 - val_loss: 1.7638 - val_acc: 0.3850
Epoch 6/100
4800/4800 [==============================] - 1s 281us/step - loss: 1.7131 - acc: 0.3996 - val_loss: 1.5578 - val_acc: 0.4700
Epoch 7/100
4800/4800 [==============================] - 1s 285us/step - loss: 1.5532 - acc: 0.4637 - val_loss: 1.3730 - val_acc: 0.

4800/4800 [==============================] - 1s 286us/step - loss: 0.0475 - acc: 0.9850 - val_loss: 1.7342 - val_acc: 0.6933
Epoch 61/100
4800/4800 [==============================] - 1s 283us/step - loss: 0.0742 - acc: 0.9777 - val_loss: 1.7246 - val_acc: 0.6875
Epoch 62/100
4800/4800 [==============================] - 1s 287us/step - loss: 0.0734 - acc: 0.9771 - val_loss: 1.8353 - val_acc: 0.6725
Epoch 63/100
4800/4800 [==============================] - 1s 293us/step - loss: 0.0795 - acc: 0.9727 - val_loss: 1.9690 - val_acc: 0.6692
Epoch 64/100
4800/4800 [==============================] - ETA: 0s - loss: 0.1012 - acc: 0.969 - 1s 283us/step - loss: 0.1018 - acc: 0.9696 - val_loss: 1.7517 - val_acc: 0.6850
Epoch 65/100
4800/4800 [==============================] - 1s 293us/step - loss: 0.0639 - acc: 0.9817 - val_loss: 1.7895 - val_acc: 0.6867
Epoch 66/100
4800/4800 [==============================] - 1s 293us/step - loss: 0.0746 - acc: 0.9788 - val_loss: 1.7113 - val_acc: 0.6808
Epoch 67/

In [12]:
from tensorflow.keras.models import load_model
best_model = load_model('models/base.h5')

In [13]:
evaluate = best_model.evaluate([valid_x, valid_x_letter], valid_y)
evaluate

48/48 [==============================] - 0s 3ms/step


[1.1084407567977905, 0.6041666666666666]

In [14]:
#submission = pd.read_csv('data/val.csv')
#submission['digit'] = np.argmax(best_model.predict([x_test, x_letter_test]), axis=1)
#submission.to_csv('data/val(CNN).csv', index=False)